<a href="https://colab.research.google.com/github/tbahng/MAR653-FinalProject/blob/jason/price_and_neighborhood.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Abstract

Find out how neighborhood can affect rental pricing

# Load Data

Data files are stored on google drive.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [4]:
import pandas as pd
import numpy as np

file_path = '/content/drive/My Drive/MAR 653/Final Project/data/{}'

# Listing data
listings_summary_df = pd.read_csv(file_path.format('listings.csv'))
listings_detail_df = pd.read_csv(file_path.format('listings.csv.gz'), compression='gzip')

# Prizm data
prizm_zipcode_df = pd.read_excel(file_path.format('prizm_attributes.xlsx'), sheet_name='zipcodes')
prizm_attributes_df = pd.read_excel(file_path.format('prizm_attributes.xlsx'), sheet_name='attributes')
prizm_zipcode_df.replace('', np.nan, inplace=True)
prizm_zipcode_df.dropna(axis=0, inplace=True)

import warnings
warnings.filterwarnings('ignore')


/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (61,62,94,95) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


# Data Transformation

The output dataframe, **listings_neighbourhood_df**, shall contain the following columns: **price**, **price_normalized**, **bedrooms**, **beds**, **upper**, **upper_middle**, **lower_middle**, and **lower**.

In [0]:
# Get price from listings_summary_df
# Get zipcode, bedrooms, and beds from listing_detail_df
import numpy as np

listings_neighbourhood_df = listings_summary_df[['price']]
listings_neighbourhood_df['zipcode'] = listings_detail_df['zipcode']
listings_neighbourhood_df['bedrooms'] = listings_detail_df['bedrooms']
listings_neighbourhood_df['beds'] = listings_detail_df['beds']

listings_neighbourhood_df.drop(9446, inplace=True)
listings_neighbourhood_df.drop(55, inplace=True)
listings_neighbourhood_df.drop(1660, inplace=True)
listings_neighbourhood_df.drop(4279, inplace=True)
listings_neighbourhood_df.drop(12377, inplace=True)
listings_neighbourhood_df.drop(15110, inplace=True)
listings_neighbourhood_df.drop(20543, inplace=True)
listings_neighbourhood_df.drop(21968, inplace=True)
listings_neighbourhood_df.drop(24371, inplace=True)
listings_neighbourhood_df.drop(25125, inplace=True)
listings_neighbourhood_df.drop(28562, inplace=True)
listings_neighbourhood_df.drop(31986, inplace=True)
listings_neighbourhood_df.drop(33587, inplace=True)
listings_neighbourhood_df.drop(39722, inplace=True)
listings_neighbourhood_df.drop(40122, inplace=True)
listings_neighbourhood_df.drop(42478, inplace=True)
listings_neighbourhood_df.drop(50491, inplace=True)
listings_neighbourhood_df.replace('', np.nan, inplace=True)
listings_neighbourhood_df.dropna(axis=0, inplace=True)

def clean_zipcode(x):
  if '\n' in x:
    return x.split('\n')[0]
  elif 'NY' in x:
    return x.split(' ')[1]
  elif '-' in x:
    return x.split('-')[0]
  else:
    return x
listings_neighbourhood_df['zipcode'] = listings_neighbourhood_df['zipcode'].apply(clean_zipcode)

In [0]:
# Normalize Price
def set_price_level(x):
  if 69 >= x:
    return 'low'
  elif 105 >= x:
    return 'low medium' 
  elif 175 >= x:
    return 'high medium'
  else:
    return 'high'

listings_neighbourhood_df['price_normalized'] = listings_neighbourhood_df['price'].apply(set_price_level)

In [0]:
# Calculate the averaged prizm score by zipcode
prizm_zipcode_df['average_score'] = -1
for i, row in prizm_zipcode_df.iterrows():
  attribute_id_arr = row['attribute_id'].split(',')
  sum = 0
  count = len(attribute_id_arr)
  for id in attribute_id_arr:
    match_row = prizm_attributes_df.loc[prizm_attributes_df['id']==int(id)]
    row_i = match_row.index[0]
    sum += match_row['score'][row_i]
  prizm_zipcode_df.loc[i, 'average_score'] = sum / count

In [0]:
# Calculate upper, upper_middle, lower_middle, lower
listings_neighbourhood_df['upper'] = 0
listings_neighbourhood_df['upper_middle'] = 0
listings_neighbourhood_df['lower_middle'] = 0
listings_neighbourhood_df['lower'] = 0

for i, row in listings_neighbourhood_df.iterrows():
  match_row = prizm_zipcode_df.loc[prizm_zipcode_df['zipcode']==int(row['zipcode'])]
  average_score = match_row['average_score'][match_row.index[0]]
  if average_score >= 4:
    listings_neighbourhood_df.loc[i, 'upper'] = 1
  elif average_score >=3:
    listings_neighbourhood_df.loc[i, 'upper_middle'] = 1
  elif average_score >= 2:
    listings_neighbourhood_df.loc[i, 'lower_middle'] = 1
  else:
    listings_neighbourhood_df.loc[i, 'lower'] = 1


In [0]:
# Drop zipcode
listings_neighbourhood_df.drop(columns=['zipcode'], inplace=True)

In [10]:
listings_neighbourhood_df.head()

,price,bedrooms,beds,price_normalized,upper,upper_middle,lower_middle,lower
0,225,0.0,1.0,high,0,1,0,0
1,89,1.0,4.0,low medium,0,1,0,0
2,200,1.0,1.0,high,0,1,0,0
3,60,1.0,1.0,low,0,1,0,0
4,79,1.0,1.0,low medium,0,1,0,0


# Cross Tabulation

Create a cross tabulation between price_normalized and neighborhood (i.e. upper, upper_middle, lower_middle, lower). The value is the sum of rentals in each neighborhood.

In [11]:
# Create cross tabulation dataframe
import pandas as pd

data_arr = []

# high
listing_sum = listings_neighbourhood_df[listings_neighbourhood_df['price_normalized'] == 'high'][['upper','upper_middle','lower_middle','lower']].sum()
data_arr.append([listing_sum['upper'], listing_sum['upper_middle'], listing_sum['lower_middle'], listing_sum['lower']])

# high medium
listing_sum = listings_neighbourhood_df[listings_neighbourhood_df['price_normalized'] == 'high medium'][['upper','upper_middle','lower_middle','lower']].sum()
data_arr.append([listing_sum['upper'], listing_sum['upper_middle'], listing_sum['lower_middle'], listing_sum['lower']])

# low medium
listing_sum = listings_neighbourhood_df[listings_neighbourhood_df['price_normalized'] == 'low medium'][['upper','upper_middle','lower_middle','lower']].sum()
data_arr.append([listing_sum['upper'], listing_sum['upper_middle'], listing_sum['lower_middle'], listing_sum['lower']])

# low
listing_sum = listings_neighbourhood_df[listings_neighbourhood_df['price_normalized'] == 'low'][['upper','upper_middle','lower_middle','lower']].sum()
data_arr.append([listing_sum['upper'], listing_sum['upper_middle'], listing_sum['lower_middle'], listing_sum['lower']])

cross_tabulation_df = pd.DataFrame(data=data_arr, index=['high', 'high_medium', 'low_medium', 'low'], columns=['upper', 'upper_middle', 'lower_middle', 'lower'])
cross_tabulation_df.head()

,upper,upper_middle,lower_middle,lower
high,2958,8947,436,0
high_medium,2215,9413,930,0
low_medium,1252,9641,1417,0
low,762,9780,2186,0


# Regression of Price vs Beds and Bedrooms (Group By Neighborhood)



### Modeling

In [59]:
# Create modeling using sklearn linear_model
from sklearn import linear_model
import numpy as np

lm = linear_model.LinearRegression()
data_dict = {}

# high
listings = listings_neighbourhood_df[listings_neighbourhood_df['upper'] == 1]
y = listings[['price']]
x = listings[['beds', 'bedrooms']]
model = lm.fit(x,y)
data_dict['upper'] = np.append(model.coef_[0], model.intercept_[0])

# upper_middle
listings = listings_neighbourhood_df[listings_neighbourhood_df['upper_middle'] == 1]
y = listings[['price']]
x = listings[['beds', 'bedrooms']]
model = lm.fit(x,y)
data_dict['upper_middle'] = np.append(model.coef_[0], model.intercept_[0])

# high
listings = listings_neighbourhood_df[listings_neighbourhood_df['lower_middle'] == 1]
y = listings[['price']]
x = listings[['beds', 'bedrooms']]
model = lm.fit(x,y)
data_dict['lower_middle'] = np.append(model.coef_[0], model.intercept_[0])

model_df = pd.DataFrame(data=data_dict, index=['bed (coef.)', 'bedrooms (coef.)', 'y-intercept'])
model_df

,upper,upper_middle,lower_middle
bed (coef.),52.607474,20.053675,22.314814
bedrooms (coef.),57.426145,56.438106,37.804581
y-intercept,54.660247,59.240152,20.648825


### Visualization

In [97]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np

x = np.arange(6)

fig = make_subplots(
    rows=1, cols=3,
    specs=[[{"type": "scatter"},{"type": 'scatter'},{'type': 'scatter'}]],
    subplot_titles=('upper','upper middle', 'lower middle')
)

bed = model_df['upper']['bed (coef.)']
bedrooms = model_df['upper']['bedrooms (coef.)']
intercept = model_df['upper']['y-intercept']
fig.add_trace(
    go.Scatter(x=x, y=bed*x+bedrooms*x+intercept),
    row=1, col=1)

bed = model_df['upper_middle']['bed (coef.)']
bedrooms = model_df['upper_middle']['bedrooms (coef.)']
intercept = model_df['upper_middle']['y-intercept']
fig.add_trace(
    go.Scatter(x=x, y=bed*x+bedrooms*x+intercept),
    row=1, col=2)

bed = model_df['lower_middle']['bed (coef.)']
bedrooms = model_df['lower_middle']['bedrooms (coef.)']
intercept = model_df['lower_middle']['y-intercept']
fig.add_trace(
    go.Scatter(x=x, y=bed*x+bedrooms*x+intercept),
    row=1, col=3)
fig.update_yaxes(range=[0,650])
fig.show()